## On-premises specific scripts.

In [ ]:

def set_gpu(gpu=0):
    """Set "CUDA_VISIBLE_DEVICES" for allocating GPU memory.

    GPU memory should be allocated before importing torch.
    """
    if gpu is None:
        gpu = 0
        mem = 1e32
        gpu_usage = get_gpu_usage()
        for _idx, _mem in gpu_usage.items():
            if _mem < mem:
                mem = _mem
                gpu = _idx

    if hasattr(gpu, "__len__"):
        gpu = ",".join([f"{_}" for _ in gpu])
    else:
        gpu = str(gpu)

    os.environ["CUDA_VISIBLE_DEVICES"] = gpu
    print(f"GPU: {gpu}")
    return gpu


def get_gpu_usage():
    from subprocess import run, PIPE

    # query = ','.join(['index', 'timestamp', 'name', 'utilization.gpu', 'utilization.memory']) # 'memory.used'])
    # cmd = f'nvidia-smi --format=csv --query-gpu={query}'
    counts = 10
    #    cmd = f'nvidia-smi stats -c {counts} -d gpuUtil,memUtil'
    cmd = f"nvidia-smi stats -c {counts} -d memUtil"
    p = run(cmd.split(), stdout=PIPE, stderr=PIPE, text=True)
    if p.returncode > 0:
        return p

    gpu = [_.split(",") for _ in p.stdout.splitlines()]
    gpu_indices = set([int(_[0]) for _ in gpu])
    gpuUtil = {_: [] for _ in gpu_indices}
    for (_i, _stat, _t, _mem) in gpu:
        gpuUtil[int(_i)].append(float(_mem))
    for _i in gpu_indices:
        gpuUtil[_i] = sum(gpuUtil[_i]) / len(gpuUtil[_i])
    return gpuUtil
